<a href="https://colab.research.google.com/github/durgesh846/COLORIZATION-OF-BLACK-WHITE-IMAGE-USING-MACHINE-LEARNING/blob/master/Copy_of_FINAL_YEAR_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Image Colorization With GANs**



In [ ]:
! pip install pyngrok

In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
port_no = 5000

In [ ]:
from flask import Flask, request, send_file, make_response
from PIL import Image
import io
import tensorflow as tf
import numpy as np
import cv2

app = Flask(__name__)
ngrok.set_auth_token("2PosU2T1Ustq6Y5WBcDynNPjlPI_2Uqerhz5kbJCeaf9T7EdM")
public_url = ngrok.connect(port_no).public_url


@app.route('/process-image', methods=['POST'])
def process_image():

    # Get the image file from the request
    image_file = request.files['image']

    # Open the image file
    image = Image.open(image_file)

    # Resize the image
    image = image.resize((200, 200))

    image = image.convert('RGB')

    # Load the generator model
    generator = tf.keras.models.load_model('/content/drive/MyDrive/mymodel/generator.h5')

    # Load the grayscale image that you want to colorize

    # Convert the PIL image to a NumPy array
    np_image = np.array(image)

    # Convert the image to grayscale
    grayscale_img = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)
    grayscale_img = cv2.resize(grayscale_img, (120, 120))
    grayscale_img = np.expand_dims(grayscale_img, axis=-1)
    grayscale_img = grayscale_img.astype(np.float32) / 255.0

    # Generate a new colored image from the grayscale input
    colored_img = generator.predict(np.array([grayscale_img]))

    # Save the processed image to a buffer
    buffer = io.BytesIO()

    colored_img = colored_img.squeeze() * 255.0
    colored_img = np.clip(colored_img, 0, 255).astype(np.uint8)
    Image.fromarray(colored_img).save(buffer, format='JPEG')


    # Return the processed image as a file
    buffer.seek(0)

    # Create a custom response object
    response = make_response(buffer.getvalue())
    response.headers['Content-Type'] = 'image/jpeg'

    # Add the Access-Control-Allow-Origin header to allow requests from any domain
    response.headers.add('Access-Control-Allow-Origin', '*')

    return response


print(f"To access the Global link please click {public_url}")
app.run(port=port_no)



## **1. Downloading and Processing the data**







In [ ]:
from google.colab import drive
drive.mount('/content/drive')


We'll now parse the images ( RGB images to be precise ) one by one, and transform each one to a grayscale image using PIL's `.convert( 'L' )` method. So our dataset will have samples of $( \ grayscale \ image \ , \ RGB \ image \ )$


In [ ]:

from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os

# The batch size we'll use for training
batch_size = 64

# Size of the image required to train our model
img_size = 120

# These many images will be used from the data archive
dataset_split = 2500

master_dir = '/content/drive/MyDrive/data'
x = []
y = []
for image_file in os.listdir( master_dir )[ 0 : dataset_split ]:
    rgb_image = Image.open( os.path.join( master_dir , image_file ) ).resize( ( img_size , img_size ) )
    # Normalize the RGB image array
    rgb_img_array = (np.asarray( rgb_image ) ) / 255
    gray_image = rgb_image.convert( 'L' )
    # Normalize the grayscale image array
    gray_img_array = ( np.asarray( gray_image ).reshape( ( img_size , img_size , 1 ) ) ) / 255
    # Append both the image arrays
    x.append( gray_img_array )
    y.append( rgb_img_array )

# Train-test splitting
train_x, test_x, train_y, test_y = train_test_split( np.array(x) , np.array(y) , test_size=0.1 )

# Construct tf.data.Dataset object
dataset = tf.data.Dataset.from_tensor_slices( ( train_x , train_y ) )
dataset = dataset.batch( batch_size )



## **2. The GAN**
First, we'll implement the generator then the discriminator and finally the loss functions required by both of them.



### **A. Generator** 




In [ ]:

def get_generator_model():

    inputs = tf.keras.layers.Input( shape=( img_size , img_size , 1 ) )

    conv1 = tf.keras.layers.Conv2D( 16 , kernel_size=( 5 , 5 ) , strides=1 )( inputs )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )
    conv1 = tf.keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1)( conv1 )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )
    conv1 = tf.keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1)( conv1 )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )

    conv2 = tf.keras.layers.Conv2D( 32 , kernel_size=( 5 , 5 ) , strides=1)( conv1 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )
    conv2 = tf.keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 )( conv2 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )
    conv2 = tf.keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 )( conv2 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )

    conv3 = tf.keras.layers.Conv2D( 64 , kernel_size=( 5 , 5 ) , strides=1 )( conv2 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )
    conv3 = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 )( conv3 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )
    conv3 = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 )( conv3 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )

    bottleneck = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' , padding='same' )( conv3 )

    concat_1 = tf.keras.layers.Concatenate()( [ bottleneck , conv3 ] )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_1 )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( conv_up_3 )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' )( conv_up_3 )

    concat_2 = tf.keras.layers.Concatenate()( [ conv_up_3 , conv2 ] )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_2 )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( conv_up_2 )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' )( conv_up_2 )

    concat_3 = tf.keras.layers.Concatenate()( [ conv_up_2 , conv1 ] )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( concat_3 )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( conv_up_1 )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 3 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu')( conv_up_1 )

    model = tf.keras.models.Model( inputs , conv_up_1 )
    return model



### **B. Discriminator**



In [ ]:

def get_discriminator_model():
    layers = [
        tf.keras.layers.Conv2D( 32 , kernel_size=( 7 , 7 ) , strides=1 , activation='relu' , input_shape=( 120 , 120 , 3 ) ),
        tf.keras.layers.Conv2D( 32 , kernel_size=( 7, 7 ) , strides=1, activation='relu'  ),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D( 64 , kernel_size=( 5 , 5 ) , strides=1, activation='relu'  ),
        tf.keras.layers.Conv2D( 64 , kernel_size=( 5 , 5 ) , strides=1, activation='relu'  ),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1, activation='relu'  ),
        tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1, activation='relu'  ),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1, activation='relu'  ),
        tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1, activation='relu'  ),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense( 512, activation='relu'  )  ,
        tf.keras.layers.Dense( 128 , activation='relu' ) ,
        tf.keras.layers.Dense( 16 , activation='relu' ) ,
        tf.keras.layers.Dense( 1 , activation='sigmoid' ) 
    ]
    model = tf.keras.models.Sequential( layers )
    return model



### **C. Loss Functions**




In [ ]:

cross_entropy = tf.keras.losses.BinaryCrossentropy()
mse = tf.keras.losses.MeanSquaredError()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output) - tf.random.uniform( shape=real_output.shape , maxval=0.1 ) , real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output) + tf.random.uniform( shape=fake_output.shape , maxval=0.1  ) , fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output , real_y):
    real_y = tf.cast( real_y , 'float32' )
    return mse( fake_output , real_y )

generator_optimizer = tf.keras.optimizers.Adam( 0.0005 )
discriminator_optimizer = tf.keras.optimizers.Adam( 0.0005 )

generator = get_generator_model()
discriminator = get_discriminator_model()



## **3. Training The GAN**


In [ ]:

@tf.function
def train_step( input_x , real_y ):
   
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate an image -> G( x )
        generated_images = generator( input_x , training=True)
        # Probability that the given image is real -> D( x )
        real_output = discriminator( real_y, training=True)
        # Probability that the given image is the one generated -> D( G( x ) )
        generated_output = discriminator(generated_images, training=True)
        
        # L2 Loss -> || y - G(x) ||^2
        gen_loss = generator_loss( generated_images , real_y )
        # Log loss for the discriminator
        disc_loss = discriminator_loss( real_output, generated_output )
    
    #tf.keras.backend.print_tensor( tf.keras.backend.mean( gen_loss ) )
    #tf.keras.backend.print_tensor( gen_loss + disc_loss )

    # Compute the gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Optimize with Adam
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))




**To start the training**

In [ ]:

num_epochs = 25

for e in range( num_epochs ):
    print( e )
    for ( x , y ) in dataset:
        # Here ( x , y ) represents a batch from our training dataset.
        print( x.shape )
        train_step( x , y )



## **4. Results**

We plot an image to see the results


In [ ]:

import matplotlib.pyplot as plt

y = generator( test_x[ 0 : 25 ] ).numpy()
i = 14
image = Image.fromarray( ( y[i] * 255 ).astype( 'uint8' ) ).resize( ( 1024 , 1024 ) )
image = np.asarray( image )
plt.imshow( image )
plt.show()


In [ ]:

image = Image.fromarray( ( test_y[i] * 255 ).astype( 'uint8' ) ).resize( ( 1024 , 1024 ) )
plt.imshow( image )
plt.show()


In [ ]:

import matplotlib.pyplot as plt

plt.imshow( test_x[i].reshape((120,120)) , cmap='gray' )
plt.show()


# Saving the Model

In [ ]:
generator.save('/content/drive/MyDrive/mymodel/generator.h5')
discriminator.save('/content/drive/MyDrive/mymodel/discriminator.h5')

# Loading the model

In [ ]:
import tensorflow as tf
generator = tf.keras.models.load_model('/content/drive/MyDrive/mymodel/generator.h5')
discriminator = tf.keras.models.load_model('/content/drive/MyDrive/mymodel/discriminator.h5')

# Generating the new coloured Image

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

# Load the generator model
generator = tf.keras.models.load_model('/content/drive/MyDrive/mymodel/generator.h5')

# Load the grayscale image that you want to colorize
grayscale_img = cv2.imread('/content/drive/MyDrive/newData/20057.jpg', cv2.IMREAD_GRAYSCALE)
grayscale_img = cv2.resize(grayscale_img, (120, 120))
grayscale_img = np.expand_dims(grayscale_img, axis=-1)
grayscale_img = grayscale_img.astype(np.float32)/255.0

# Generate a new colored image from the grayscale input
colored_img = generator.predict(np.array([grayscale_img]))



# Loading the generated coloured image

In [ ]:
import matplotlib.pyplot as plt

# Convert BGR image to RGB
colored_img = cv2.cvtColor(colored_img[0], cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
plt.imshow(colored_img)
plt.axis('off')
plt.show()
